In [1]:
import os
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm # 반복문의 진행상황 확인하는 모듈
import time
from scipy.spatial import distance

# 경고끄기 (option)
import warnings
warnings.filterwarnings('ignore')

In [2]:
model_url = "https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1"

IMAGE_SHAPE = (224, 224)
model_name = 'R50x1_object'
category = 'bag'

layer = hub.KerasLayer(model_url, input_shape=IMAGE_SHAPE+(3,))
model = tf.keras.Sequential([layer])
model.summary()

Metal device set to: Apple M1 Pro


2022-09-28 16:26:29.388081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-28 16:26:29.388595: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
Total params: 23,500,352
Trainable params: 0
Non-trainable params: 23,500,352
_________________________________________________________________


In [3]:
from pathlib import Path
p = str(Path(os.getcwd()).parents[1])

image_path =  p + f"/crops/{category}/"
output_path = p + f"/vector_frame/{category}/"
# print(p)
# print(os.path.exists(output_path))

if not os.path.exists(output_path):
    os.mkdir(output_path[:output_path.find(category)] + '/' + category)

In [4]:
print(image_path)
print(output_path)

/Users/iseongmin/workspaces/project2/crops/bag/
/Users/iseongmin/workspaces/project2/vector_frame/bag/


In [7]:
array = extract('/Users/iseongmin/Desktop/지갑.jpeg')

1/1 [==============================] - 0s 102ms/step


In [10]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [17]:
np.set_printoptions(precision=10)
np.set_printoptions(suppress=True)

In [19]:
from pprint import pprint

In [20]:
pprint(array)

array([ 0.034376003 ,  0.          ,  1.8119482   ,  0.27187583  ,
        0.          ,  2.1136112   ,  0.          ,  0.          ,
        0.          ,  0.          ,  0.          ,  0.5004003   ,
       11.794234    ,  0.02478816  ,  2.0043907   ,  6.30561     ,
        0.          ,  0.45954505  ,  0.          ,  0.61616325  ,
        9.133765    ,  0.24757677  ,  0.          ,  0.9041846   ,
        3.0939975   ,  9.519011    ,  3.5657656   ,  0.10596569  ,
        0.6078538   ,  0.06431472  ,  0.          ,  0.14696859  ,
        0.06184426  ,  1.7103341   ,  1.111343    ,  0.          ,
        0.          ,  0.          ,  0.31583354  ,  2.4616394   ,
       12.476034    ,  0.          ,  0.031245109 ,  0.08390533  ,
        2.8088496   ,  1.9829376   ,  3.3260345   ,  2.444489    ,
        0.3587415   ,  0.079719305 ,  0.07606711  ,  3.178125    ,
        0.35454318  ,  0.          ,  0.14897162  ,  0.29121053  ,
        4.257559    ,  0.          ,  0.24265853  ,  2.3668144

In [5]:
def extract(file):
    file = Image.open(file).convert('RGB').resize((224, 224))
    file = np.array(file)/255.0 # 정규화

    embedding = model.predict(file[np.newaxis, ...])
    feature_np = np.array(embedding)
    flattened_feature = feature_np.flatten()

    return flattened_feature

In [6]:
def make_dataframe(category=category, model_name=model_name):
    global image_path
    global output_path
    file_list = os.listdir(image_path)
    file_list_img = [file for file in file_list if file.endswith(".png") or file.endswith(".jpeg") or file.endswith(".jpg")]
    tmp_df = pd.DataFrame()
    for i, img in enumerate(tqdm(file_list_img)):
        output = extract(image_path+'/'+img)
        tmp_df = tmp_df.append({'filename':img, 'output':output}, ignore_index=True)

    np.save(output_path+f'{category}_filename({model_name}).npy', tmp_df['filename'])
    np.save(output_path+f'{category}_output({model_name}).npy', tmp_df['output']) # np.array배열 형태로 나중에 불러오기 위해 따로 저장
    return tmp_df

In [7]:
tmp_df = make_dataframe()

  0%|          | 0/2097 [00:00<?, ?it/s]

2022-09-16 17:22:15.627347: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-16 17:22:15.744222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 58ms/step


In [8]:
tmp_df

,filename,output
0,V0000244H07290008-1.jpg,"[0.24041007, 0.0, 0.7117855, 0.0, 0.3743473, 0..."
1,V0000366H07080004-1.jpg,"[0.20050395, 0.0, 3.7767928, 2.2973394, 0.0536..."
2,F2022032800000615-1.jpg,"[0.0, 0.0, 0.39821148, 1.224729, 0.42873064, 0..."
3,V0000428H06300008-1.jpg,"[0.024019504, 0.0, 4.689266, 0.12153283, 0.059..."
4,F2022050200001181-1.jpg,"[0.0107280575, 0.0, 2.7740784, 0.0, 0.6782091,..."
...,...,...
2092,F2022081900003415-1.jpg,"[1.6519744, 0.0, 8.294312, 0.23065682, 0.40387..."
2093,F2022070500001301-1.jpg,"[1.722608, 0.0, 2.9996006, 0.0, 0.5324247, 1.4..."
2094,V0002034H05290002-1.jpg,"[0.0, 0.0, 2.9096608, 0.11057483, 0.0, 1.74078..."
2095,F2022062700002287-1.jpg,"[0.35511997, 0.0, 0.16387163, 0.0, 0.0, 7.6797..."


In [9]:
def get_dataframe(category=category, model_name=model_name):
    global output_path    
    tmp_filename = np.load(output_path+f'{category}_filename({model_name}).npy', allow_pickle=True)
    tmp_output = np.load(output_path+f'{category}_output({model_name}).npy', allow_pickle=True)
    df = pd.DataFrame({'filename':tmp_filename, 'output':tmp_output})
    return df

In [10]:
get_dataframe()

,filename,output
0,V0000244H07290008-1.jpg,"[0.24041007, 0.0, 0.7117855, 0.0, 0.3743473, 0..."
1,V0000366H07080004-1.jpg,"[0.20050395, 0.0, 3.7767928, 2.2973394, 0.0536..."
2,F2022032800000615-1.jpg,"[0.0, 0.0, 0.39821148, 1.224729, 0.42873064, 0..."
3,V0000428H06300008-1.jpg,"[0.024019504, 0.0, 4.689266, 0.12153283, 0.059..."
4,F2022050200001181-1.jpg,"[0.0107280575, 0.0, 2.7740784, 0.0, 0.6782091,..."
...,...,...
2092,F2022081900003415-1.jpg,"[1.6519744, 0.0, 8.294312, 0.23065682, 0.40387..."
2093,F2022070500001301-1.jpg,"[1.722608, 0.0, 2.9996006, 0.0, 0.5324247, 1.4..."
2094,V0002034H05290002-1.jpg,"[0.0, 0.0, 2.9096608, 0.11057483, 0.0, 1.74078..."
2095,F2022062700002287-1.jpg,"[0.35511997, 0.0, 0.16387163, 0.0, 0.0, 7.6797..."


In [ ]:
def get_cos_sim(file, category=category, metric='cosine'):
    before_time = time.time()
    file2vec = extract(file) # 이미지 벡터화
    df = get_dataframe() # 데이터프레임 가져오기
    df = df.append({'filename':file, 'output':file2vec}, ignore_index=True)
    
    cos_sim_array = np.zeros((len(df)))
    for i in range(0, len(df)):
        cos_sim_array[i] = distance.cdist([file2vec] , [df.iloc[i, 1]], metric)[0] # 벡터화된 이미지 기준
    df['cos_sim']=cos_sim_array
    after_time = time.time()
    runtime = after_time-before_time
    return df, runtime # 런타임 비교용

In [ ]:
def show_sim(input_file):
    global image_path
    global output_path
    cos_sim_df, runtime = get_cos_sim(input_file)
    df_top_sim = cos_sim_df.sort_values(by='cos_sim')[:15]

    # 그래프 그리는 부분은 서비스 시 생략 가능
    f, ax = plt.subplots(3, 5, figsize=(40, 20))

    for i in range(len(df_top_sim)):
        if i == 0: 
            tmp_img = Image.open(df_top_sim.iloc[i, 0]).convert('RGB')
            title = f'Original \n{df_top_sim.iloc[i, 0]}'
        else : 
            tmp_img = Image.open(image_path+'/'+df_top_sim.iloc[i, 0]).convert('RGB')
            title = f'similarity no.{i} \n{df_top_sim.iloc[i, 0]}'

        sim = f'cos : {df_top_sim.iloc[i, 2]:.3f}' 
        ax[i//5][i%5].imshow(tmp_img, aspect='auto')
        ax[i//5][i%5].set_title(title, pad=20, size=25) # 5열짜리 표를 만드는 것이므로 단순히 5로 나눈 나머지와 몫을 사용한 것임
        ax[i//5][i%5].annotate(sim, (0,10), fontsize=18, color='red')
    
    print(f'소요시간 : {runtime:.3f}')
    plt.show()

In [ ]:
def show_sim_threshold(input_file, threshold, savefig=False):
    global image_path
    global output_path
    cos_sim_df, runtime = get_cos_sim(input_file)
    df_top_sim = cos_sim_df[cos_sim_df.cos_sim <= threshold].sort_values(by='cos_sim')[:30]
    # 그래프 그리는 부분은 서비스 시 생략 가능
    if len(df_top_sim) <= 10:
        f, ax = plt.subplots(2, 5, figsize=(40, 20))
    elif len(df_top_sim) <=15:
        f, ax = plt.subplots(3, 5, figsize=(40, 30))
    elif len(df_top_sim) <=20:
        f, ax = plt.subplots(4, 5, figsize=(40, 40))
    elif len(df_top_sim) <=25:
        f, ax = plt.subplots(5, 5, figsize=(40, 45))
    else:
        f, ax = plt.subplots(6, 5, figsize=(40, 50))

    for i in range(len(df_top_sim)):
        if i == 0: 
            tmp_img = Image.open(df_top_sim.iloc[i, 0]).convert('RGB')
            title = f'Original \n{df_top_sim.iloc[i, 0]}'
        else : 
            tmp_img = Image.open(image_path+'/'+df_top_sim.iloc[i, 0]).convert('RGB')
            title = f'similarity no.{i} \n{df_top_sim.iloc[i, 0]}'
        
        sim = f'cos : {df_top_sim.iloc[i, 2]:.3f}' 
        ax[i//5][i%5].imshow(tmp_img, aspect='auto')
        ax[i//5][i%5].set_title(title, pad=20, size=25) # 5열짜리 표를 만드는 것이므로 단순히 5로 나눈 나머지와 몫을 사용한 것임
        ax[i//5][i%5].annotate(sim, (0,10), fontsize=18, color='red')
    
    if savefig:
        global model_name
        plt.savefig(f'./test_result/{model_name}_test_result_{time.strftime("%H_%M_%S")}.jpg', pad_inches=0)
    print(f'소요시간 : {runtime:.3f}')
    plt.show()

In [ ]:
get_dataframe()
input_file_path = os.listdir(os.getcwd())
input_files = [file for file in input_file_path if file.endswith(".png") or file.endswith(".jpeg") or file.endswith(".jpg")]



In [ ]:
threshold = 0.4 # 임계값은 변경해가면서 확인 가능
show_sim_threshold('test_car.png', threshold=threshold, savefig=False)
# show_sim_threshold(input_file, threshold=threshold, savefig=True)
# savefig = True 인자 넣으면 결과 사진 현재 디렉토리에 저장

In [ ]:
show_sim_threshold(input_files[1], threshold=threshold, savefig=True)

In [ ]:
show_sim_threshold(input_files[2], threshold=threshold, savefig=True)

In [ ]:
show_sim_threshold(input_files[3], threshold=threshold, savefig=True)